In [139]:
import cv2, sys
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageEnhance, ImageChops
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [154]:
import os
import datetime
import time, random
from tqdm import tqdm
import glob

In [141]:
# 테스트 데이터 꺼내오기 
# 독립문 훈련, 테스트셋 나누기 
door_folder = "C:/source/image/indep_door"
park_folder = "C:/source/image/tapgol_park"
door_file_list = [x for x in os.listdir(door_folder)]
park_file_list = [x for x in os.listdir(park_folder)]

In [142]:
door_test_list = door_file_list[:200]
len(door_test_list)

200

In [143]:
door_train_list = door_file_list[200:]
len(door_train_list)

136

In [144]:
# 독립문 테스트셋
file_name_freq  = 0
file_name_freq += 1 
for i in tqdm(range(len(door_test_list))):
    img = load_img("C:/source/image/indep_door/" + door_test_list[i])
    img = img.resize((300, 300))
    img.save("C:/source/image/test/indep_door/" + door_test_list[i])

100%|███████████████████████████████████████████████████████████████████████████| 200/200 [00:14<00:00, 14.08it/s]


In [145]:
# 독립문 훈련셋
for i in tqdm(range(len(door_train_list))):
    img = load_img("C:/source/image/indep_door/" + door_train_list[i])
    img = img.resize((300, 300))
    img.save("C:/source/image/train/indep_door/" + door_train_list[i])
    

100%|███████████████████████████████████████████████████████████████████████████| 136/136 [00:10<00:00, 12.69it/s]


In [146]:
park_test_list = park_file_list[:200]
park_train_list = park_file_list[200:]

In [147]:
# 탑골공원 테스트셋
for i in tqdm(range(len(park_test_list))):
    img = load_img("C:/source/image/tapgol_park/" + park_test_list[i])
    img = img.resize((300, 300))
    img.save("C:/source/image/test/tapgol_park/" + park_test_list[i])

100%|███████████████████████████████████████████████████████████████████████████| 200/200 [00:19<00:00, 10.16it/s]


In [148]:
# 탑골공원 훈련셋
for i in tqdm(range(len(park_train_list))):
    img = load_img("C:/source/image/tapgol_park/" + park_train_list[i])
    img = img.resize((300, 300))
    img.save("C:/source/image/train/tapgol_park/" + park_train_list[i])

100%|███████████████████████████████████████████████████████████████████████████| 605/605 [00:57<00:00, 10.61it/s]


In [185]:
# 학습이미지 부풀리기
datagen = ImageDataGenerator(\
                            rotation_range=10, # 정수범위만큼 무작위 회전
                            width_shift_range=0.1,  # 픽셀단위로 좌우이동
                            height_shift_range=0.1, # 픽셀단위로 상하이동
                            shear_range=0.5, # 이미지 찌그러트리기
                            zoom_range=0.2, # 이미지 확대 축소 
                            horizontal_flip=True, # 좌우반전
                            fill_mode="nearest") # 이미지에 여백 생겼을 때

In [186]:
# 학습이미지만 부풀리기
# 독립문 
file_name_freq = 0
for i in tqdm(range(len(door_train_list))):
    img = load_img("C:/source/image/train/indep_door/" + door_train_list[i])  # PIL 이미지
    x = img_to_array(img) # (3, 300, 300) 크기의 Numpy 배열
    x = x.reshape((1,) + x.shape) # (1, 3, 300, 300) 크기의 Numpy 배열
    file_name_freq += 1
    
    # 변환된 이미지를 배치 단위로 생성해서 지정된 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,\
                             save_to_dir="C:/source/image/train/indep_door/",\
                             save_prefix=door_train_list[i] + str(file_name_freq),\
                             save_format="jpg"):
        i += 1
        if i > 6:
            break # 이미지 한 개 당 6개씩 부풀리기

  0%|                                                                                     | 0/136 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:/source/image/train/indep_door/HF010435_0103_0089.JPG'

In [192]:
# # 학습이미지만 부풀리기
# # 탑골공원 
# file_name_freq = 0
# for i in tqdm(range(len(park_train_list))):
#     img = load_img("C:/source/image/train/tapgol_park/" + park_train_list[i])  # PIL 이미지
#     x = img_to_array(img) # (3, 300, 300) 크기의 Numpy 배열
#     x = x.reshape((1,) + x.shape) # (1, 3, 300, 300) 크기의 Numpy 배열
#     file_name_freq += 1
    
#     # 변환된 이미지를 배치 단위로 생성해서 지정된 폴더에 저장
#     i = 0
#     for batch in datagen.flow(x, batch_size=1,\
#                              save_to_dir="C:/source/image/train/tapgol_park/",\
#                              save_prefix=park_train_list[i] + str(file_name_freq),\
#                              save_format="jpg"):
#         i += 1
#         if i > 2:
#             break # 이미지 한 개 당 6개씩 부풀리기

In [193]:
# 데이터 훈련시키기 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics


In [194]:
b_size=1

In [195]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('C:/source/image/train/',target_size=(300,300),
                                                    batch_size=b_size, class_mode='categorical')

Found 2000 images belonging to 2 classes.


In [196]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('C:/source/image/test/',target_size=(300,300),
                                                    batch_size=b_size, class_mode='categorical')

Found 400 images belonging to 2 classes.


In [202]:
model = Sequential()
model.add(Conv2D(15, (3, 3), input_shape=(300,300,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(15, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(30, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(30))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [203]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 298, 298, 15)      420       
_________________________________________________________________
activation_30 (Activation)   (None, 298, 298, 15)      0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 149, 149, 15)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 147, 147, 15)      2040      
_________________________________________________________________
activation_31 (Activation)   (None, 147, 147, 15)      0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 73, 73, 15)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 71, 71, 30)       

In [204]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=0.0002),\
              metrics=['accuracy'])

In [205]:
steps_train = len(train_generator)
steps_test = len(test_generator)

In [206]:
history = model.fit(
       train_generator,
       steps_per_epoch=steps_train,
       epochs=10,
       validation_data=test_generator,
       validation_steps=steps_test )

Epoch 1/10
2000/2000 [==============================] - 242s 121ms/step - loss: 0.1016 - accuracy: 0.9525 - val_loss: 0.0621 - val_accuracy: 0.9825
Epoch 2/10
2000/2000 [==============================] - 246s 123ms/step - loss: 0.0381 - accuracy: 0.9845 - val_loss: 0.1191 - val_accuracy: 0.9850
Epoch 3/10
2000/2000 [==============================] - 266s 133ms/step - loss: 0.0196 - accuracy: 0.9865 - val_loss: 0.1404 - val_accuracy: 0.9825
Epoch 4/10
2000/2000 [==============================] - 234s 117ms/step - loss: 0.0124 - accuracy: 0.9940 - val_loss: 0.1793 - val_accuracy: 0.9750
Epoch 5/10
2000/2000 [==============================] - 209s 104ms/step - loss: 0.0046 - accuracy: 0.9970 - val_loss: 0.2212 - val_accuracy: 0.9775
Epoch 6/10
2000/2000 [==============================] - 193s 96ms/step - loss: 0.0087 - accuracy: 0.9970 - val_loss: 0.2694 - val_accuracy: 0.9775
Epoch 7/10
2000/2000 [==============================] - 180s 90ms/step - loss: 0.0022 - accuracy: 0.9985 - val_lo